In [1]:
import napari
import numpy as np
from skimage import data

import napari_plot
from napari_plot._qt.qt_viewer import QtViewer
from dask_image.imread import imread
from magicgui.widgets import ComboBox, Container, PushButton, SpinBox, FileEdit, FloatSpinBox, Label, TextEdit

from dask_image import imread
import pandas as pd
import dask.array as da
import os
from napari.qt.threading import thread_worker

from dipy.align.imaffine import AffineMap
from numba import jit
import tifffile as tf





C:\Users\zwartlab-users\.conda\envs\neuroviewer\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class neuro_viewer(object):
    
    im = None
    denoised_dir = None
    volume_df = None
    coms_df = None
    con_df = None
    dff_tidy = None
    
    
    def __init__(self, fr = 3.07):
        
        

        self.viewer = napari.Viewer()
        self.viewer.add_image(np.zeros((512, 512)), name = "deconvolved", opacity = 0.5)
        self.im_subset = self.viewer.layers[0]
        self.im_subset.colormap = "inferno"
        
        self.viewer.add_image(np.zeros((512, 512)), name = "overview", opacity = 0.5)
        self.overview_im = self.viewer.layers[1]
        self.overview_im.colormap = "cyan"
        
        self.viewer.add_image(np.zeros((512, 512)), name = "reference", opacity = 0.5)
        self.reference_brain = self.viewer.layers[2]
        
        self.viewer.add_image(np.zeros((512, 512)), name = "template", opacity = 0.5)
        self.template = self.viewer.layers[3]
        self.template.colormap = "magenta"
        
       
        
        
        self.stim_icon = self.viewer.add_shapes(np.array([[0, 0, 0],
                                                           [0, 0, 0],
                                                           [0, 0, 0],
                                                           [0, 0, 0]]), shape_type='rectangle', edge_width=5,
                                                            edge_color='#55ff00', face_color = "#55ff00", name = "stim")
        #self.all_rois = self.viewer.add_shapes(shape_type = "polygon", edge_width=0.2,
        #                  edge_color='white', face_color='transparent', ndim = 4)
        self.select_roi = self.viewer.add_shapes(shape_type='polygon', edge_width=0.2,
                          edge_color='white', face_color='transparent', ndim = 4, name = "rois")
        
        self.im_subset.contrast_limits = (0, 255)
        self.overview_im.contrast_limits = (0, 10000)
        self.fr = fr
        
        
        # create label menus for stim, trial, angle, velocity
        #update choices as required when stim is changed
        self.stim_menu = ComboBox(label='Stim', choices = ["visual", "motion", "flow"], tooltip = "Select stim")
        self.trial_menu = ComboBox(label='Trial', choices = [], tooltip = "Select stim")
        self.angle_menu = ComboBox(label='Angle', choices = [], tooltip = "Select stim")
        self.velocity_menu = ComboBox(label='Velocity', choices = [], tooltip = "Select stim")
        
        
        dropdown_widget = Container(widgets=[self.stim_menu, self.trial_menu, self.angle_menu, self.velocity_menu])
        
        
        self.stim_menu.changed.connect(self.stim_changed)
        self.trial_menu.changed.connect(self.trial_changed)
        self.angle_menu.changed.connect(self.angle_changed)
        self.velocity_menu.changed.connect(self.velocity_changed)
        
        
        
        
        self.cell_ids = SpinBox(label = "cell_id", tooltip = "change cell")
        cell_widget = Container(widgets=[self.cell_ids])
        
        
        
        
        self.volume_df_picker= FileEdit(value='./Select volume_df.h5', tooltip = "Select volume_df")
        file_widget = Container(widgets=[self.volume_df_picker])
        
        
        registration_label = Label(name = "Registration", label = "Registration")
        self.moving_im_file = FileEdit(value='./Select image to register', tooltip = "Select template tif")
        self.reference_im_file = FileEdit(value='./Select reference image', tooltip = "Select reference tif")
                                   
        self.translate_x = FloatSpinBox(label = "translate x", tooltip = "change x translation", min = -500, max = 500, value = 0)
        self.translate_y = FloatSpinBox(label = "translate y", tooltip = "change y translation", min = -500, max = 500, value = 0)
        self.translate_z = FloatSpinBox(label = "translate z", tooltip = "change z translation", min = -500, max = 500, value = 0)
        self.affine_name = TextEdit(label = "affine filename", value = "affine.npy")
        self.save_affine_button = PushButton(label = "save affine")
        
        self.translate_x.changed.connect(self.translate)
        self.translate_y.changed.connect(self.translate)
        self.translate_z.changed.connect(self.translate)
        self.save_affine_button.clicked.connect(self.save_starting_affine)
        
        self.moving_im_file.changed.connect(self.read_moving_im)
        self.reference_im_file.changed.connect(self.read_reference_im)
        
        translation_widget = Container(widgets=[registration_label,
                                                self.moving_im_file, self.reference_im_file,
                                                self.translate_x, self.translate_y, self.translate_z, 
                                                self.affine_name, self.save_affine_button])
        
        
        
        
        self.volume_df_picker.changed.connect(self.volume_df_picker_changed)
        self.viewer.window.add_dock_widget(dropdown_widget)
        self.viewer.window.add_dock_widget(cell_widget)
        self.viewer.window.add_dock_widget(file_widget)
        self.viewer.window.add_dock_widget(translation_widget)
        self.add_dff_widget()
        self.add_ephys_widget()
        self.cell = self.cell_ids.value
        self.cell_ids.changed.connect(self.plot_cell)
        self.select_roi.events.highlight.connect(self.roi_selected)
        
        self.io_affine = None
        self.grid_copy = None
        
        
      
    def load_image(self):
        """Load image"""
        if self.volume_subset.shape[0] > 0:
            print("loading dask images")
            self.im_subset.data = self.im[:, self.first_vol:self.end_vol ].compute()
            print("images loaded")
            
        else:
            print("no volumes")
        pass
    
    def load_volume_df(self, path):
        """Load pandas dataframe containing all information about volume"""
        print("loading volume_df")
        self.volume_df = pd.read_hdf(path)
        
    def load_coms_df(self, path):
        print("loading coms_df")
        self.coms_df = pd.read_hdf(path)
        
    
    def load_con_df(self, path):
        print("loading con_df")
        self.con_df = pd.read_hdf(path)
        self.plot_all_contours()
        
    def load_dff_df(self, path):
        """Load all dff data"""
        print("loading dff_df")
        self.dff_tidy = pd.read_hdf(path, "df")
    
    
    def lazy_load_images(self):
        
        print("loading images")
        # load all images as one dask array - dask.array.stack(data = [],  axis = 0)
        #files = os.listdir(self.denoised_dir)
        #ims = [os.path.join(self.denoised_dir,file) for file in files if '16bit' in file]
        
        #da_ims = []

        #for im in ims:
        #    da_im = imread.imread(im, nframes = 1)
        #    da_ims.append(da_im)
        self.im  = da.from_zarr(os.path.join(self.denoised_dir, "8bit.zarr"), chunks= (5, 1000, -1, -1))
        
        #self.im = da.stack(da_ims)
        
        print("loading finished")
        
    
    def stim_changed(self, event):
        print(self.volume_df.trial.unique())
        self.stim = event
        self.stim_subset  = self.volume_df[self.volume_df.stim == self.stim]
        self.update_menu_choices()
        self.load_subset()
        print(event)
        
    def load_subset(self):
        self.get_volume_subset()
        if self.volume_subset.shape[0] > 0:
            self.load_image()
            self.plot_cell(self.cell)
            self.label_stim()
            self.plot_ephys()
            
        
        
    def update_menu_choices(self):
        trials = self.stim_subset.trial.dropna().unique().tolist()
        angles = self.stim_subset.angle.dropna().unique().tolist()
        velocities = self.stim_subset.velocity.dropna().unique().tolist()
        
        self.angle = angles[0]
        self.velocity = velocities[0]
        self.trial = trials[0]
        
        # initiates change loop
        self.trial_menu.choices = trials
        self.angle_menu.choices = angles
        self.velocity_menu.choices = velocities
        
        
        print(self.angle, self.velocity, self.trial)
        # update choices of trial, angle, velocity
        
        
    def trial_changed(self, event):
        self.trial = event
        self.load_subset()
        print("new trial {}".format(event))
        
    def angle_changed(self, event):
        self.angle = event
        self.load_subset()
        
        print("new angle {}".format(event))
        
    def velocity_changed(self, event):
        self.velocity = event
        self.load_subset()
        print("new velocity {}".format(event))
        
    def volume_df_picker_changed(self, event):
        print(event)
        
        try:
            self.volume_df_path = event
        except:
            self.volume_df_path = event.value
        
        
        # get denoised dir
        self.denoised_dir = os.path.dirname(self.volume_df_path)
        
        # load all dataframes
        
        self.load_volume_df(self.volume_df_path)
        
        # load images
        self.lazy_load_images()
        # load reference images, and create io template to load domain and codomain grid2worlds and register
        self.load_overview_image()
        self.load_h2b_reference()
        self.create_IO_template()
        
        # load centre of masses and contours and apply any transforms
        self.load_coms_df(os.path.join(self.denoised_dir, "all_neuron_centers.h5"))
        self.load_con_df(os.path.join(self.denoised_dir, "all_neuron_contours.h5"))
        #self.load_dff_df(os.path.join(self.denoised_dir, "dff.h5"))
        
        
        
        
        self.stim_changed(self.stim_menu.value)
        
        
    def get_volume_subset(self):
        print(self.stim, self.angle, self.velocity, self.trial)
        stim_filter = self.volume_df.stim == self.stim
        angle_filter = self.volume_df.angle == self.angle
        velocity_filter = self.volume_df.velocity == self.velocity
        trial_filter = self.volume_df.trial == self.trial
        
        self.volume_subset = self.volume_df[(stim_filter) & (angle_filter) & ( velocity_filter) & (trial_filter)]
        
        if self.volume_subset.shape[0] > 0:
         # 4 seconds before
            vols_before = int(4 * self.fr)
            
            self.cond_start  = self.volume_subset.nVol.iloc[0] 
            self.cond_end = self.volume_subset.nVol.iloc[-1] 
            self.first_vol = self.cond_start - vols_before
            self.end_vol = self.cond_end + vols_before
            self.vols = np.arange(self.first_vol, self.end_vol)
            
    def add_dff_widget(self):
        
        self.viewer1d = napari_plot.ViewerModel1D()
        widget = QtViewer(self.viewer1d)
        self.viewer.window.add_dock_widget(widget, area="bottom", name="Dff Widget")
        self.viewer1d.axis.x_label = "Time"
        self.viewer1d.axis.y_label = "DeltaF/F"
        self.viewer1d.reset_view()
        
    def add_ephys_widget(self):
        
        self.viewer1d_ephys = napari_plot.ViewerModel1D()
        widget = QtViewer(self.viewer1d_ephys)
        self.viewer.window.add_dock_widget(widget, area="bottom", name="Ephys Widget", allowed_areas = ["bottom"], add_vertical_stretch = True)
        self.viewer1d_ephys.axis.x_label = "Time"
        self.viewer1d_ephys.axis.y_label = "Power"
        self.viewer1d_ephys.reset_view()
    
    def clear_plot(self):
        self.viewer1d.clear_canvas()
        self.viewer1d_ephys.clear_canvas()
    
    def plot_ephys(self):
        if self.stim != None:
            vol_filter = self.dff_tidy.nVol.isin(self.vols)
            cell_filter = self.dff_tidy.cell_id == self.cell
            trial_subset = self.dff_tidy.loc[ vol_filter & cell_filter, ["Rightconvolved", "Leftconvolved"]]
            trial_subset
            t = np.arange(trial_subset.shape[0])/ self.fr
            self.viewer1d_ephys.add_line(np.c_[t, trial_subset.Rightconvolved.to_numpy()], color = "magenta")
            self.viewer1d_ephys.add_line(np.c_[t, trial_subset.Leftconvolved.to_numpy()], color = "cyan")
            self.viewer1d_ephys.reset_view()
        
    def plot_cell(self, event):
        if self.stim != None:
            self.clear_plot()
            # define cell event
            # get dff info for cell
            # get contour of cell
            self.cell = event

            stim_filter = self.dff_tidy.stim == self.stim
            angle_filter = self.dff_tidy.angle == self.angle
            velocity_filter = self.dff_tidy.velocity == self.velocity
            trial_filter = self.dff_tidy.trial == self.trial
            cell_filter = self.dff_tidy.cell_id == self.cell


            self.dff_subset  = self.dff_tidy[(stim_filter) & (angle_filter) & (velocity_filter) & (cell_filter)]

            # plot every trial but highlight current one with thicker line

            for trial in self.dff_subset.trial.unique():
                trial_subset = self.dff_subset[self.dff_subset.trial == trial]
                trial_start = trial_subset.nVol.iloc[0]
                trial_end = trial_subset.nVol.iloc[-1]
                vols_before = int(4 * self.fr)
                first_vol = trial_start - vols_before
                last_vol = trial_end + vols_before
                vols = np.arange(first_vol, last_vol)
                vol_filter = self.dff_tidy.nVol.isin(vols)

                plotting_subset = self.dff_tidy[cell_filter & vol_filter]
                t = np.arange(0, plotting_subset.shape[0]/self.fr, 1/self.fr)

                if trial == self.trial:

                    self.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], color = "magenta")
                    regions = [
                            ([t[trial_start-first_vol], t[trial_end-first_vol]], "vertical"),
                        ]

                    layer = neuroviewer.viewer1d.add_region(
                            regions,
                            color=["green"],
                            opacity = 0.4,
                            name = "Stim",
                        )

                else:
                    self.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], name="trial {}".format(trial), color="gray")



                self.viewer1d.reset_view()
            
        
    def label_stim(self):
        start = self.cond_start - self.first_vol
        end = self.cond_end - self.first_vol
        stim_duration = end - start

        nframes = stim_duration
        centre_rs = np.tile(np.array([0, 30, 30]), (4, 1))
        centres = np.stack([centre_rs] * nframes)
        frame_pos = np.tile(np.arange(start, end).T, (4, 1)).T
        centres[:, :, 0] = frame_pos
        add_array = np.array([[0, -10, -10],
                                      [0, -10, 10],
                                      [0, 10, 10],
                                      [0, 10, -10]])

        #define boxes by adding to centre_rs
        boxes = centres + add_array.reshape(-1, *add_array.shape)
        labels = ["stim"] * stim_duration
        properties = {
            'label': labels,

        }

        # specify the display parameters for the text
        text_params = {
            'text': '{label}',
            'size': 12,
            'color': 'green',
            'anchor': 'upper_left',
            'translation': [-3, 0]}

        self.stim_icon.data = boxes
        self.stim_icon.properties = properties
        self.stim_icon.text = text_params
        
    def plot_all_contours(self): # slow
        
        #try:
        if self.io_affine is not None:
            io_affine = self.io_affine

        elif self.grid_copy is not None:
            io_affine = self.grid_copy

        else:
            io_affine = np.eye(4)

        self.transform_contours(io_affine, self.io_grid2world, self.ref_grid2world)
            
        #except:
        #    print("error registering contours")
        
        z_index = [] 
        shapes= []
        for cell in self.con_df_copy.cell_id.sort_values().unique():
            subset = self.con_df_copy[self.con_df_copy.cell_id == cell].dropna()
            contours = subset[["z","cell_id", "y", "x"]]

            contours.cell_id = np.zeros(contours.shape[0])


            z_index.append(subset.iloc[0, 2])
            
            shapes.append(np.round_(contours.to_numpy(), 2))
            
        
        self.select_roi.selected_data = set(range(self.select_roi.nshapes))
        self.select_roi.remove_selected()

        self.select_roi.add(shapes,shape_type = "polygon", edge_width=0.4,
                                  edge_color='white', face_color='transparent')
        
    
    def roi_selected(self, event):
        if len(self.select_roi.selected_data) > 0:
            self.cell_ids.value = list(self.select_roi.selected_data)[0]
            
    def load_overview_image(self):
        overview_dir = os.path.join(os.path.dirname(self.denoised_dir), "overview")
        overview_file = [os.path.join(overview_dir, file) for file in os.listdir(overview_dir) if "overview" in file][0]
        self.overview_im.data  = da.rot90(da.rot90(imread.imread(overview_file), axes = (1, 2)), axes = (1, 2))
        
    def load_h2b_reference(self):
        self.reference_brain.data = da.rot90(
                                    da.rot90(
                                    da.flip(imread.imread(r"Z:\Pierce\Elavl3-H2BRFP\Elavl3-H2BRFP_rotated.tif"), 
                                            axis=0), axes = (1, 2)), axes = (1, 2))
        
    def load_rigid_registered_overview(self):
        # load registered whole brain
        pass
    
    def load_nonrigid_registered_overview(self):
        # load registered whole brain
        pass
        
    def create_IO_template(self):
        # load registered template IO to get rough alignment 
        self.template_orig = self.im[:, :10].max(axis= 1).compute()
        
        # in future create these grid2worlds using image metadata
        self.ref_grid2world = np.array([[ 1.,    0.,         0.,          0.],
                                         [0.,   0.01171,   0.,            0.],
                                         [0.,   0.,         0.01171,      0.],
                                         [0.,   0.,         0.,           1.]])
        
        self.io_grid2world = np.array( [[ 5.,   0.,         0.,          0.],
                                         [0.,   0.008758,  0.,            0.],
                                         [0.,   0.,         0.008758,     0.],
                                         [0.,   0.,         0.,          1.]])
        
        affine = np.eye(4)
        
        
        self.template.data = self.apply_tranform(self.ref_grid2world, self.io_grid2world, affine, self.overview_im.data, self.template_orig)
        
    def apply_tranform(self, ref_grid2world, im_grid2world, affine, ref, im):
    
        
        affine_map = AffineMap(affine,
                               ref.shape, ref_grid2world,
                               im.shape, im_grid2world)
        resampled = affine_map.transform(im)
        
        return resampled
        
        
    def translate(self, event):
        
        if self.moving_grid2world is not None:
            # manually estimate translation
            self.x = self.translate_x.value
            self.y = self.translate_y.value
            self.z = self.translate_z.value

            self.grid_copy = np.eye(4)
            self.grid_copy[2, -1] += self.x # x
            self.grid_copy[1, -1] += self.y # y
            self.grid_copy[0, -1] += self.z # z
            
            if len(self.overview_im.data.shape) > 2: # if not still empty array
                self.template.data = self.apply_tranform(self.ref_grid2world, self.moving_grid2world, self.grid_copy, self.overview_im.data, self.template_orig)
                
            else:
                self.template.data = self.apply_tranform(self.ref_grid2world, self.moving_grid2world, self.grid_copy, self.reference_brain.data, self.template_orig)
            
            

    def save_starting_affine(self):
        #save affine transform
        filename = self.affine_name.value
        np.save(os.path.join(self.denoised_dir, filename), self.grid_copy)
        
        
    def load_affine_matrix(self):
        self.io_affine = np.load(os.path.join(self.denoised_dir, "io_affine.npy"))
        
    
    def transform_coordinates(self, coordinates, affine_transform, domain_transform, codomain_transform):
        # in shape z, y, x, 1
        new_coords = np.zeros(coordinates.shape)
        for coord_idx, coord in enumerate(coordinates):

            new_coord = np.dot(np.linalg.inv(codomain_transform), np.dot(affine_transform, np.dot(domain_transform, coord)))
            new_coords[coord_idx] = new_coord

        return new_coords
    
    def transform_contours(self, io_affine, domain_grid2world, codomain_grid2world): # self.io_grid2world, self.ref_grid2world
        self.con_df_copy = self.con_df.copy()
        contours = self.con_df_copy.loc[:, ["z", "y", "x"]].to_numpy()
        contour_coords = np.concatenate([contours, np.ones((contours.shape[0], 1))], axis= 1)
        new_coords = self.transform_coordinates(contour_coords, io_affine, domain_grid2world, codomain_grid2world)
        self.con_df_copy.loc[:, ["z", "y", "x"]] = new_coords[:, :3]
        
    def read_moving_im(self, event):
        # read template image
        # read image
        # assign to template layer data
        # load or create template_grid2world from image metadata
        filename = str(self.moving_im_file.value)
        
        
        if "io" in filename:
            # dont rotate
            self.moving_im = imread.imread(str(filename))
            self.moving_grid2world = np.eye(4)
            
            self.moving_grid2world[0, 0] = 5
            
            # get x, y px2ref from image metadata
            x, y = self.get_px2ref(filename)
            self.moving_grid2world[1, 1] = y
            self.moving_grid2world[2, 2] = x
            self.moving_grid2world = np.array( [[ 5.,   0.,         0.,          0.],
                                                 [0.,   0.008758,  0.,            0.],
                                                 [0.,   0.,         0.008758,     0.],
                                                 [0.,   0.,         0.,          1.]])
        else:
            # all other cases rotate
            self.moving_im = da.rot90(da.rot90(imread.imread(filename), axes = (1, 2)), axes = (1, 2))
            self.moving_grid2world = np.eye(4)
        self.template_orig = self.moving_im.compute()
        self.template.data = self.template_orig
        
        self.denoised_dir = os.path.dirname(filename)
    
    def read_reference_im(self, event):
        # read reference image
        # assign to reference layer data
        # load or create ref_grid2world from image metadata
        filename = str(self.reference_im_file.value)
        
        
        
        if "Elav" in filename:
            
            # flip and rotate 180
            self.ref_im = da.rot90(
                                    da.rot90(
                                    da.flip(imread.imread(filename), 
                                            axis=0), axes = (1, 2)), axes = (1, 2))
            
            self.ref_grid2world = np.array( [[ 2.,   0.,         0.,          0.],
                                                 [0.,   1.,  0.,            0.],
                                                 [0.,   0.,         1.,     0.],
                                                 [0.,   0.,         0.,          1.]])
            self.reference_brain.data = self.ref_im.compute()
        elif "overview" in filename:
            # rotate 180
            self.ref_im = da.rot90(da.rot90(imread.imread(filename), axes = (1, 2)), axes = (1, 2))
            
            
            self.ref_grid2world = np.array([[ 1.,    0.,         0.,          0.],
                                             [0.,   0.01171,   0.,            0.],
                                             [0.,   0.,         0.01171,      0.],
                                             [0.,   0.,         0.,           1.]])
            
            self.overview_im.data = self.ref_im.compute()
            
    def get_px2ref(self, filename):
        with tf.TiffFile(filename) as tif:
            roi_artist = tif.pages[0].tags["Artist"]
            metadata = roi_artist.value.split("\n")
            px2ref_row = [datum for datum in metadata if "pixelToRef" in datum][0]
            row_idx = metadata.index(px2ref_row)
            x = metadata[row_idx +1]
            y = metadata[row_idx +2]
            x_scale = float(x.replace(" ", "").replace("[", "").replace("]", "").split(",")[0])
            y_scale = float(y.replace(" ", "").replace("[", "").replace("]", "").split(",")[1])
            
            return x_scale, y_scale

        
        


        
        
            

    
        
        
    
        
    

In [10]:
neuroviewer = neuro_viewer()
# to do - try only setting the shapes at a later stage

In [12]:
neuroviewer.affine_name.value

'io_starting_affine.npy'

In [20]:
filename = neuroviewer.affine_name.value
np.save(os.path.join(neuroviewer.denoised_dir, filename), neuroviewer.grid_copy)
filename

'io_starting_affine.npy'

In [17]:
neuroviewer.denoised_dir = os.path.dirname(str(neuroviewer.moving_im_file.value))

In [58]:
len(neuroviewer.overview_im.data.shape)

2

In [8]:
neuroviewer.reference_brain.data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [57]:
neuroviewer.ref_im

dask.array<transpose, shape=(138, 1406, 621), dtype=uint16, chunksize=(1, 1406, 621), chunktype=numpy.ndarray>

In [ ]:
import tifffile as tf

In [ ]:
filename2 = r"Z:\Pierce\IOMultiSensory\20220427\fish1\overview\fish1_full_stack_overview_z1_steps250_00001.tif"

In [ ]:
with tf.TiffFile(filename2) as tif:
    roi_artist = tif.pages[0].tags["Artist"]

In [ ]:
tif.pages[0].tags["ResolutionUnit"].value.value

In [ ]:
(18874/1073741824) * 1000

In [ ]:
tif.pages[0].tags["XResolution"].value

In [ ]:
[print(tag) for tag in tif.pages[0].tags]

In [ ]:
metadata = tif.pages[0].tags["Software"].value.split("\n")
#metadata

In [ ]:
[print(data.split("=")[-1]) for data in metadata if "micron" in data]

In [ ]:
split_position = axes_position.replace("[", "").replace("]", "")[1:].split(" ")

In [ ]:
axes_x = float(split_position[0])
axes_x

In [ ]:
for datum in metadata:
    
    if "SI.hFastZ.position" in datum:
        fastz_position = float(datum.split("=")[-1])
    
    if "SI.hMotors.axesPosition" in datum:
        axes_position = datum.split("=")[-1]
        split_position = axes_position.replace("[", "").replace("]", "")[1:].split(" ")
        axes_x = float(split_position[0])
        axes_y = float(split_position[1])
        axes_z = float(split_position[2])
        

In [100]:
def get_px2ref(filename):
    with tf.TiffFile(filename) as tif:
        roi_artist = tif.pages[0].tags["Artist"]
        metadata = roi_artist.value.split("\n")
        px2ref_row = [datum for datum in metadata if "pixelToRef" in datum][0]
        row_idx = metadata.index(px2ref_row)
        x = metadata[row_idx +1]
        y = metadata[row_idx +2]
        x_scale = float(x.replace(" ", "").replace("[", "").replace("]", "").split(",")[0])
        y_scale = float(y.replace(" ", "").replace("[", "").replace("]", "").split(",")[1])
        return x_scale, y_scale

In [101]:
get_px2ref(filename2)

(0.01171875, 0.01171875)

In [ ]:
[print(tag) for tag in tif.pages[0].tags]

In [ ]:
#[print(tag.value) for tag in tif.pages[0].tags]

In [ ]:
metadata = roi_artist.value.split("\n")
#metadata

In [ ]:
ref_affine = np.load(os.path.join(os.path.dirname(neuroviewer.denoised_dir), "overview\\affine_map.npy"))
ref_affine

In [ ]:
zbrain_grid2world = np.array([[2.,    0.,     0.,    0.],
                             [0.,     1.,    0.,    0.],
                             [0.,     0.,     1.,   0.],
                             [0.,     0.,     0.,   1.]])


ref_grid2zbrain = np.array([[1.,   0.,      0.,    0.],
                             [0.,     1.,     0.,    200.],
                             [0.,     0.,     1.,    -200.],
                             [0.,     0.,     0.,     1.]]) 

In [ ]:
transformed_ref = neuroviewer.apply_tranform(zbrain_grid2world, ref_grid2zbrain, ref_affine, neuroviewer.reference_brain.data, neuroviewer.overview_im.data.compute())
transformed_ref.shape

In [ ]:
neuroviewer.overview_im.data = transformed_ref

In [ ]:
neuroviewer.reference_brain.data.shape

In [ ]:
neuroviewer.overview_im.data

In [ ]:
"Z:\Pierce\IOMultiSensory\20220427\fish1\overview\affine_map.npy"

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

In [ ]:
fig, ax1 = plt.subplots()
dims = neuroviewer.template.data.shape[1:]
patches = []

for cell in neuroviewer.con_df.cell_id.unique():
        contour = neuroviewer.con_df.loc[neuroviewer.con_df.cell_id == cell, ["x", "y"]].to_numpy()
        p = Polygon(contour, closed=True, edgecolor = 'white', linewidth = 0.2)

        patches.append(p)
ax1.imshow(np.zeros((dims[1], dims[0])), cmap = "gray")
collection = PatchCollection(patches, alpha=0.9)
collection.set_edgecolors("gray")
collection.set_linewidth(0.4)


ax1.add_collection(collection)

patches = []

for cell in neuroviewer.con_df_copy.cell_id.unique():
        contour = neuroviewer.con_df_copy.loc[neuroviewer.con_df_copy.cell_id == cell, ["x", "y"]].to_numpy()
        p = Polygon(contour, closed=True, edgecolor = 'white', linewidth = 0.2)

        patches.append(p)
ax1.imshow(np.zeros((dims[1], dims[0])), cmap = "gray")
collection = PatchCollection(patches, alpha=0.9)
collection.set_edgecolors("magenta")
collection.set_linewidth(0.4)


ax1.add_collection(collection)

In [ ]:
#io_affine = np.eye(4)
#io_affine[1, -1] = 5

# seems to use the absolute version of the matrix
io_affine = np.abs(np.load(os.path.join(neuroviewer.denoised_dir, "io_affine.npy")))
neuroviewer.transform_contours(io_affine, neuroviewer.io_grid2world, neuroviewer.ref_grid2world)
io_affine

In [ ]:
fig, ax1 = plt.subplots()
dims = neuroviewer.template.data.shape[1:]
patches = []

for cell in neuroviewer.con_df.cell_id.unique():
        contour = neuroviewer.con_df.loc[neuroviewer.con_df.cell_id == cell, ["x", "y"]].to_numpy()
        p = Polygon(contour, closed=True, edgecolor = 'white', linewidth = 0.2)

        patches.append(p)
ax1.imshow(np.zeros((dims[1], dims[0])), cmap = "gray")
collection = PatchCollection(patches, alpha=0.9)
collection.set_edgecolors("gray")
collection.set_linewidth(0.4)


ax1.add_collection(collection)

patches = []

for cell in neuroviewer.con_df_copy.cell_id.unique():
        contour = neuroviewer.con_df_copy.loc[neuroviewer.con_df_copy.cell_id == cell, ["x", "y"]].to_numpy()
        p = Polygon(contour, closed=True, edgecolor = 'white', linewidth = 0.2)

        patches.append(p)
ax1.imshow(np.zeros((dims[1], dims[0])), cmap = "gray")
collection = PatchCollection(patches, alpha=0.9)
collection.set_edgecolors("magenta")
collection.set_linewidth(0.4)


ax1.add_collection(collection)

In [ ]:
neuroviewer.load_dff_df(r"C:\Users\pierc\Desktop\dff.h5")

In [ ]:
neuroviewer.create_IO_template()

In [ ]:
np.save(os.path.join(neuroviewer.denoised_dir, "io_starting_affine.npy"), neuroviewer.grid_copy)

In [ ]:
neuroviewer.plot_all_contours()


In [ ]:
neuroviewer.select_roi.selected_data = set(range(neuroviewer.select_roi.nshapes))

In [ ]:
neuroviewer.select_roi.nshapes

In [ ]:
neuroviewer.con_df_copy.head()

In [ ]:
if neuroviewer.io_affine is not None:
    io_affine = neuroviewer.io_affine

elif neuroviewer.grid_copy is not None:
    io_affine = neuroviewer.grid_copy

else:
    io_affine = np.eye(4)

io_affine

In [ ]:
io_affine[1, -1] = -1
io_affine

In [ ]:
io_affine = np.abs(io_affine)
io_affine

In [ ]:
io_affine

In [ ]:
np.round_(io_affine)

In [ ]:
neuroviewer.transform_contours(io_affine, neuroviewer.io_grid2world, neuroviewer.io_grid2world)

In [ ]:
np.round_(neuroviewer.con_df_copy[neuroviewer.con_df_copy.cell_id == 0].to_numpy(), 2)

In [ ]:
neuroviewer.con_df.head(20)

In [ ]:
shapes= []

for cell in neuroviewer.con_df_copy.cell_id.sort_values().unique():
    subset = neuroviewer.con_df_copy[neuroviewer.con_df_copy.cell_id == cell].dropna()
    contours = subset[["z","cell_id", "y", "x"]]

    contours.cell_id = np.zeros(contours.shape[0])

    shapes.append(contours.to_numpy())


    
#neuroviewer.select_roi.selected_data = set(range(neuroviewer.select_roi.nshapes))
#neuroviewer.select_roi.remove_selected()
neuroviewer.select_roi.add(shapes, shape_type = "polygon", edge_width=0.4,
                          edge_color='white', face_color='transparent')

In [ ]:
shapes[0]

In [ ]:
neuroviewer.select_roi.nshapes

In [ ]:
neuroviewer.select_roi.data[0]


In [ ]:
neuroviewer.grid_copy, neuroviewer.io_grid2world, neuroviewer.ref_grid2world

In [ ]:
neuroviewer.template_orig

In [ ]:
np.save(os.path.join(self.denoised_dir, "io_starting_affine.npy"), self.grid_copy)

In [ ]:
neuroviewer.select_roi.edgewidt

In [ ]:
def transform_coordinates(coordinates, affine_transform, domain_transform, codomain_transform):
    # in shape z, y, x, 1
    new_coords = np.zeros(coordinates.shape)
    for coord_idx, coord in enumerate(coordinates):
        
        new_coord = np.dot(np.linalg.inv(codomain_transform), np.dot(affine_transform, np.dot(domain_transform, coord)))
        new_coords[coord_idx] = new_coord
        
    return new_coords

In [ ]:
%%timeit
transform_coordinates(contour_coords, neuroviewer.grid_copy)

In [ ]:
%%timeit
transform_coordinates(contour_coords, neuroviewer.grid_copy)

In [ ]:
#con_df_copy = neuroviewer.con_df.copy()
new_coords = transform_coordinates(contour_coords, io_affine, neuroviewer.ref_grid2world, neuroviewer.io_grid2world)
con_df_copy.loc[:, ["z", "y", "x"]] = new_coords[:, :3]

In [ ]:
con_df_copy

In [ ]:
# map to grid
io_affine = np.load(os.path.join(neuroviewer.denoised_dir, "IO_affine.npy"))
io_affine

In [ ]:
neuroviewer.template.data.shape #z,  y, x

In [ ]:
io_affine2 = np.eye(4)
io_affine2[:, -1] = io_affine[:, -1]
io_affine2

In [ ]:
io_grid2world = np.eye(4)
io_grid2world[:, :-1] = neuroviewer.io_grid2world[:, :-1]
io_grid2world

In [ ]:
contour_coords

In [ ]:
neuroviewer.ref_grid2world

In [ ]:
transform_matrix = io_grid2world @ (io_affine2@neuroviewer.ref_grid2world)

In [ ]:
transform_matrix

In [ ]:
neuroviewer.con_df.head(), con_df_copy.head()

In [ ]:
neuroviewer.con_df

In [ ]:
neuroviewer.grid_copy

In [ ]:
def apply_tranform(ref_grid2world, im_grid2world, ref, im):
    
        
        identity = np.eye(4)
        affine_map = AffineMap(identity,
                               ref.shape, ref_grid2world,
                               im.shape, im_grid2world)
        resampled = affine_map.transform(im)
        
        return resampled, affine_map.affine

In [ ]:
neuroviewer.overview_im.data

In [ ]:
neuroviewer.template_orig = neuroviewer.im[:, :1000].max(axis= 1).compute()

In [ ]:
neuroviewer.template_orig.shape

In [ ]:
apply_tranform(neuroviewer.ref_grid2world, neuroviewer.io_grid2world, neuroviewer.overview_im.data, neuroviewer.template_orig)

In [ ]:
neuroviewer.plot_cell(0)

In [ ]:
overview_dir = os.path.join(os.path.dirname(neuroviewer.denoised_dir), "overview")
overview_file = [os.path.join(overview_dir, file) for file in os.listdir(overview_dir)][0]
neuroviewer.overview_im  = imread.imread(overview_file)

In [ ]:
ref_brain = neuroviewer.reference_brain.data.compute()

In [ ]:
ref_brain3 = neuroviewer.reference_brain.data.flip(axis=0)

In [ ]:
ref_brain3 = da.flip(neuroviewer.reference_brain.data, axis=0)

In [ ]:
ref_brain2 = ref_brain.copy()

In [ ]:
ref_brain2.shape

In [ ]:
ref_brain2 = np.flip(ref_brain2, axis=0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2)
ax1.imshow(ref_brain3[0])
ax2.imshow(ref_brain3[-1])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib widget

In [ ]:
fig, ax = plt.subplots()
ax.imshow(neuroviewer.reference_brain[0])

In [ ]:
neuroviewer.overview_im

In [ ]:
vol_filter = neuroviewer.dff_tidy.nVol.isin(neuroviewer.vols)
cell_filter = neuroviewer.dff_tidy.cell_id == neuroviewer.cell
trial_subset = neuroviewer.dff_tidy.loc[ vol_filter & cell_filter, ["Rightconvolved", "Leftconvolved"]]
trial_subset
t = np.arange(trial_subset.shape[0])/ neuroviewer.fr
neuroviewer.viewer1d_ephys.add_line(np.c_[t, trial_subset.Rightconvolved.to_numpy()], color = "magenta")
neuroviewer.viewer1d_ephys.add_line(np.c_[t, trial_subset.Leftconvolved.to_numpy()], color = "cyan")

In [ ]:
neuroviewer.select_roi.events.highlight.connect(highlight)

In [ ]:
def highlight(event):
    neuroviewer.cell_ids.value = list(neuroviewer.select_roi.selected_data)[0]
    print(neuroviewer.select_roi.selected_data)
    neuroviewer.select_roi.current_edge_color = "green"
    neuroviewer.select_roi.current_edge_width = 1


In [ ]:
neuroviewer.cell_ids.value = list(neuroviewer.select_roi.selected_data)[0]

In [ ]:
%%timeit
neuroviewer.im[:, 100:120].compute()

In [ ]:
files = os.listdir(neuroviewer.denoised_dir)
ims = [os.path.join(neuroviewer.denoised_dir,file) for file in files if '16bit' in file]

da_ims = []

for im in ims:
    da_im = imread.imread(im)
    da_ims.append(da_im)

neuroviewer.im = da.stack(da_ims)

In [ ]:
da_im = da.from_zarr(os.path.join(neuroviewer.denoised_dir, "8bit.zarr"), chunks= (5, 1000, -1, -1))

In [ ]:
da_im

In [ ]:
neuroviewer.im

In [ ]:
neuroviewer.im_subset.contrast_limits

In [ ]:
neuroviewer.im_subset.data = neuroviewer.im[:, neuroviewer.first_vol: neuroviewer.end_vol]

In [ ]:
import time

In [ ]:
start = time.time()
da_im[:, neuroviewer.first_vol : neuroviewer.end_vol ].compute()
end = time.time()
end - start

In [ ]:
da_im[:, neuroviewer.first_vol : neuroviewer.end_vol]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib widget

In [ ]:
fig, ax = plt.subplots()
ax.imshow(da_im[0, 0])

In [ ]:
files = os.listdir(neuroviewer.denoised_dir)
im_path = [os.path.join(neuroviewer.denoised_dir,file) for file in files if '16bit' in file][0]



neuroviewer.im = imread.imread(im_path)
        

In [ ]:
neuroviewer.im # file saved incorrectly - save as 16 bit instead and in big tiff format

In [ ]:
centre_rs = np.tile(np.array([0, 30, 30]), (4, 1))
centres = np.stack([centre_rs] * nframes)
frame_pos = np.tile(np.arange(start, end).T, (4, 1)).T

In [ ]:
neuroviewer.volume_subset.shape

In [ ]:
# loop through cells get x, y contour for each #
# add first axis for volumes
# get z_index
start = neuroviewer.first_vol
end = neuroviewer.end_vol
nframes = end - start


z_index = [] 
shapes= []
for cell in neuroviewer.con_df.cell_id.unique():
    subset = neuroviewer.con_df[neuroviewer.con_df.cell_id == cell].dropna()
    contours = subset[["z","cell_id", "y", "x"]]
    
    contours.cell_id = np.zeros(contours.shape[0])

    
    z_index.append(subset.iloc[0, 2])
    shapes.append(contours.to_numpy())
    

In [ ]:
len(z_index), len(shapes)

In [ ]:
neuroviewer.viewer.add_shapes([cell_roi], shape_type='polygon', edge_width=0.2,
                          edge_color='white', face_color='transparent', ndim = 4)

In [ ]:
neuroviewer.select_roi = neuroviewer.viewer.layers[-1]

In [ ]:
neuroviewer.viewer.layers[-1]

In [ ]:
cell_roi

In [ ]:
neuroviewer.select_roi.add(cell_roi)

In [ ]:
cell_roi.shape

In [ ]:
neuroviewer.select_roi.data

In [ ]:

#neuroviewer.all_rois.data = shapes
#neuroviewer.all_rois.edge_color = "white"
#neuroviewer.all_rois.face_color = "transparent"

neuroviewer.viewer.add_shapes(shapes, shape_type='polygon', edge_width=0.2,
                          edge_color='white', face_color='transparent', z_index = z_index)

In [ ]:


stim_duration

In [ ]:
start =12
end = 24
stim_duration = end - start

In [ ]:
start = neuroviewer.cond_start - neuroviewer.first_vol
end = neuroviewer.cond_end - neuroviewer.first_vol
stim_duration = end - start

nframes = stim_duration
centre_rs = np.tile(np.array([0, 30, 30]), (4, 1))
centres = np.stack([centre_rs] * nframes)
frame_pos = np.tile(np.arange(start, end).T, (4, 1)).T
centres[:, :, 0] = frame_pos
add_array = np.array([[0, -10, -10],
                              [0, -10, 10],
                              [0, 10, 10],
                              [0, 10, -10]])

#define boxes by adding to centre_rs
boxes = centres + add_array.reshape(-1, *add_array.shape)
labels = ["stim"] * stim_duration
properties = {
    'label': labels,

}

# specify the display parameters for the text
text_params = {
    'text': 'label: {label}',
    'size': 12,
    'color': 'green',
    'anchor': 'upper_left',
    'translation': [-3, 0]}

neuroviewer.stim_icon.data = boxes
neuroviewer.stim_icon.properties = properties
neuroviewer.stim_icon.text = text_params



In [ ]:
neuroviewer.stim_icon.data

In [ ]:
dff = pd.read_hdf(r"C:\Users\pierc\Desktop\dff.h5")
dff.head()

In [ ]:
neuroviewer.dff_tidy.head()

In [ ]:
stim_filter = neuroviewer.dff_tidy.stim == neuroviewer.stim
angle_filter = neuroviewer.dff_tidy.angle == neuroviewer.angle
velocity_filter = neuroviewer.dff_tidy.velocity == neuroviewer.velocity
trial_filter = neuroviewer.dff_tidy.trial == neuroviewer.trial
cell_filter = neuroviewer.dff_tidy.cell_id == neuroviewer.cell

In [ ]:
dff_df = neuroviewer.dff_tidy

In [ ]:
#neuroviewer.dff_tidy == dff_df
vol_filter = neuroviewer.dff_tidy.nVol.isin(neuroviewer.vols)
vol_filter

In [ ]:
regions = [
    ([0, 1], "vertical"),
]

layer = neuroviewer.viewer1d.add_region(
    regions,
    color=[white],
    opacity = 0.5,
    name = "Stim",
)

In [ ]:
neuroviewer.dff_subset = neuroviewer.dff_tidy[stim_filter & angle_filter & velocity_filter & cell_filter]
neuroviewer.dff_subset.head()

In [ ]:
neuroviewer.dff_subset.shape

In [ ]:
np.arange(0, plotting_subset.shape[0]/neuroviewer.fr, 1/neuroviewer.fr)

In [ ]:
plotting_subset.shape

In [ ]:
t.shape

In [ ]:
for trial in neuroviewer.dff_subset.trial.unique():
    trial_subset = neuroviewer.dff_subset[neuroviewer.dff_subset.trial == trial]
    trial_start = trial_subset.nVol.iloc[0]
    trial_end = trial_subset.nVol.iloc[-1]
    vols_before = int(4 * neuroviewer.fr)
    first_vol = trial_start - vols_before
    last_vol = trial_end + vols_before
    vols = np.arange(first_vol, last_vol)
    vol_filter = neuroviewer.dff_tidy.nVol.isin(vols)
    
    plotting_subset = neuroviewer.dff_tidy[cell_filter & vol_filter]
    t = np.arange(0, plotting_subset.shape[0]/neuroviewer.fr, 1/neuroviewer.fr)

    if trial == neuroviewer.trial:
        
        neuroviewer.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], color = "magenta")

    else:
        neuroviewer.viewer1d.add_line(np.c_[t, plotting_subset.dff.to_numpy()], name="trial {}".format(trial), color="gray")
        
    neuroviewer.viewer1d.reset_view()

In [ ]:
plotting_subset.shape, t.shape

In [ ]:
trial_start = trial_subset.nVol.iloc[0]
trial_end = trial_subset.nVol.iloc[-1]
vols_before = int(4 * neuroviewer.fr)
first_vol = trial_start - vols_before
last_vol = trial_end + vols_before
vols = np.arange(first_vol, last_vol)
vol_filter = neuroviewer.dff_tidy.nVol.isin(vols)

plotting_subset = neuroviewer.dff_tidy[cell_filter & vol_filter]

In [ ]:
plotting_subset

In [ ]:
np.c_[trial_subset.timebyepoch.to_numpy(), trial_subset.dff.to_numpy()]

In [ ]:
trial_subset.timebyepoch.to_numpy()

In [ ]:
trial_subset.dff.to_numpy()

In [ ]:
neuroviewer.velocity = 1.0

stim_filter = neuroviewer.volume_df.stim == neuroviewer.stim
angle_filter = neuroviewer.volume_df.angle == neuroviewer.angle
velocity_filter = neuroviewer.volume_df.velocity == neuroviewer.velocity
trial_filter = neuroviewer.volume_df.trial == neuroviewer.trial

neuroviewer.volume_subset = neuroviewer.volume_df[(stim_filter) & (angle_filter) & ( velocity_filter) & (trial_filter)]
neuroviewer.vols = neuroviewer.volume_subset.nVol
neuroviewer.vols

In [ ]:
neuroviewer.trial = 0 

In [ ]:
neuroviewer.volume_df[(neuroviewer.volume_df.stim == self.stim)]

In [ ]:
dff_df = pd.read_hdf(os.path.join(neuroviewer.denoised_dir, "dff.h5"))

In [ ]:
dff_df.set_index("cell_id", inplace = True)
dff_df.head()

In [ ]:
neuroviewer.dff_tidy.T.to_hdf(os.path.join(neuroviewer.denoised_dir, "dff_test.h5"), "df")

In [ ]:
neuroviewer.dff_tidy.T.shape

In [ ]:
neuroviewer.load_dff_df()

In [ ]:
neurovi

In [ ]:
neuroviewer.im
napari.run()

In [ ]:
#viewer = napari.Viewer()



viewer1d = napari_plot.ViewerModel1D()
widget = QtViewer(viewer1d)
viewer.window.add_dock_widget(widget, area="bottom", name="Line Widget")

# example data
x = np.arange(0.1, 4, 0.1)
y1 = np.exp(-1.0 * x)
y2 = np.exp(-0.5 * x)

# example variable error bar values
y1err = 0.1 + 0.1 * np.sqrt(x)
y2err = 0.1 + 0.1 * np.sqrt(x / 2)


viewer1d.add_line(np.c_[x, y1], name="Line 1", color="lightblue")
viewer1d.add_centroids(
    np.c_[x, y1 - y1err, y1 + y1err], orientation="vertical", color="lightblue", opacity=0.5, name="Line 1 (errors)"
)
viewer1d.add_line(np.c_[x, y2], name="Line 2", color="orange")
viewer1d.add_centroids(
    np.c_[x, y2 - y2err, y2 + y2err], orientation="vertical", color="orange", opacity=0.5, name="Line 2 (errors)"
)
viewer1d.camera.extent = (-0.1, 4.1, 1.0, -0.3)
viewer1d.axis.x_label = ""
viewer1d.axis.y_label = ""
viewer1d.reset_view()

napari.run()

In [ ]:
neuroviewer.add_dff_widget()